In [1]:
import numpy as np
import operator
import os

In [2]:
def createDataSet():
    group = np.array([[1.0, 1.1], [1.0, 1.0], [0, 0], [0, 0.1]])
    labels = ['A', 'A', 'B', 'B']
    return group, labels

In [3]:
createDataSet()

(array([[1. , 1.1],
        [1. , 1. ],
        [0. , 0. ],
        [0. , 0.1]]), ['A', 'A', 'B', 'B'])

In [4]:
# 取前k个距离最近的点
def classify0(inX, dataSet, labels, k):
    m = dataSet.shape[0]
    data = np.tile(inX, (m, 1)) - dataSet
    data = data ** 2
    # 计算x^2 + y^2
    dist = data.sum(axis = 1)
    dist = dist ** (1 / 2)
    dist_arg = dist.argsort()
    m = {}
    for i in range(k):
        m[labels[dist_arg[i]]] = m.get(labels[dist_arg[i]], 0) + 1
    classCnt = sorted(m.items(), key = operator.itemgetter(1))
    return classCnt[-1][0]  

In [5]:
group, labels = createDataSet()
classify0([0, 0], group, labels, 3)

'B'

In [6]:
inX = [0, 1]
np.tile(inX, (3, 2))

array([[0, 1, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 1]])

In [7]:
# 读取文件内容
def file2matrix(filename):
    fr = open(filename)
    lines = fr.readlines()
    n = len(lines)
    data = np.zeros((n, 3))
    label = []
    index = 0
    for line in lines:
        line.strip()
        nums = line.split("\t")
        data[index] = nums[0:3]
        label.append(int(nums[-1]))
        index += 1
    return data, label

In [8]:
data, label = file2matrix('datingTestSet2.txt')
print(data)
print(label)

[[4.0920000e+04 8.3269760e+00 9.5395200e-01]
 [1.4488000e+04 7.1534690e+00 1.6739040e+00]
 [2.6052000e+04 1.4418710e+00 8.0512400e-01]
 ...
 [2.6575000e+04 1.0650102e+01 8.6662700e-01]
 [4.8111000e+04 9.1345280e+00 7.2804500e-01]
 [4.3757000e+04 7.8826010e+00 1.3324460e+00]]
[3, 2, 1, 1, 1, 1, 3, 3, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 2, 1, 3, 1, 3, 1, 2, 1, 1, 2, 3, 3, 1, 2, 3, 3, 3, 1, 1, 1, 1, 2, 2, 1, 3, 2, 2, 2, 2, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 3, 1, 2, 3, 2, 2, 1, 3, 1, 1, 3, 3, 1, 2, 3, 1, 3, 1, 2, 2, 1, 1, 3, 3, 1, 2, 1, 3, 3, 2, 1, 1, 3, 1, 2, 3, 3, 2, 3, 3, 1, 2, 3, 2, 1, 3, 1, 2, 1, 1, 2, 3, 2, 3, 2, 3, 2, 1, 3, 3, 3, 1, 3, 2, 2, 3, 1, 3, 3, 3, 1, 3, 1, 1, 3, 3, 2, 3, 3, 1, 2, 3, 2, 2, 3, 3, 3, 1, 2, 2, 1, 1, 3, 2, 3, 3, 1, 2, 1, 3, 1, 2, 3, 2, 3, 1, 1, 1, 3, 2, 3, 1, 3, 2, 1, 3, 2, 2, 3, 2, 3, 2, 1, 1, 3, 1, 3, 2, 2, 2, 3, 2, 2, 1, 2, 2, 3, 1, 3, 3, 2, 1, 1, 1, 2, 1, 3, 3, 3, 3, 2, 1, 1, 1, 2, 3, 2, 1, 3, 1, 3, 2, 2, 3, 1, 3, 1, 1, 2, 1, 2, 2, 1, 

In [9]:
import matplotlib
import matplotlib.pyplot as plt
figure = plt.figure()
# 111指画图的长宽尺寸
ax = figure.add_subplot(111)
# ax.scatter(data[:, 1], data[:, 2])
ax.scatter(data[:, 0], data[:, 1], 15.0 * np.array(label), 15.0 * np.array(label))
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
# 归一化数据
def autoNorm(data):
    minn = data.min(axis = 0)
    maxx = data.max(axis = 0)
    ranges = maxx - minn
    res = np.zeros(data.shape)
    m = data.shape[0]
    res = data - np.tile(minn, (m, 1))
    res = res / np.tile(ranges, (m, 1))
    return res, ranges, minn

In [11]:
res, ranges, minVals = autoNorm(data)
print(res)

[[0.44832535 0.39805139 0.56233353]
 [0.15873259 0.34195467 0.98724416]
 [0.28542943 0.06892523 0.47449629]
 ...
 [0.29115949 0.50910294 0.51079493]
 [0.52711097 0.43665451 0.4290048 ]
 [0.47940793 0.3768091  0.78571804]]


In [12]:
# 1 : 9 测试
def datingClassTest():
    testRatio = 0.10
    data, label = file2matrix('datingTestSet2.txt')
    norm, ranges, minVals = autoNorm(data)
    m = data.shape[0]
    testNum = int(testRatio * m)
    errorCNT = 0
    for i in range(testNum):
        res = classify0(norm[i, :], norm[testNum:m, :], label[testNum : m], 3)
#         print(f"the classifier came back with {res}, the real answer is {label[i]}")
        if res != label[i]:
            errorCNT += 1
    print(f"the total error rate is {errorCNT * 1.0 / m}")

In [13]:
datingClassTest()

the total error rate is 0.005


In [14]:
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    percentTats = float(input("percentage of playing vedio games?"))
    ffMiles = float(input('frequent flier miles per year?'))
    iceCream = float(input('liters of ice cream consumed per year?'))
    data, label = file2matrix('datingTestSet2.txt')
    norm, ranges, minVals = autoNorm(data)
    arr = np.array([ffMiles, percentTats, iceCream])
    res = classify0((arr - minVals) / ranges, norm, label, 3)
    print(f"you will probably like this person : {resultList[res - 1]}")

In [15]:
classifyPerson()

percentage of playing vedio games?100
frequent flier miles per year?100
liters of ice cream consumed per year?100
you will probably like this person : in large doses


In [16]:
def img2vector(filename):
    fr = open(filename)
    res = np.zeros((1, 1024))
    for i in range(32):
        line = fr.readline()
        for j in range(32):
            res[0, i * 32 + j] = int(line[j])
    return res

In [17]:
def headWritingClassfier():
    trainPath = './digits/trainingDigits'
    validPath = './digits/testDigits'
    dataName = os.listdir(trainPath)
    n = len(dataName)
    trainlabel = []
    traindata = np.zeros((n, 1024))
    for i in range(n):
        fileName = dataName[i].split('.')[0]
        trainlabel.append(int(fileName.split('_')[0]))
        traindata[i, :] = img2vector(trainPath + "/" +  dataName[i])
        
    testName = os.listdir(validPath)
    errorCnt = 0
    m = len(testName)
    for i in range(m):
        fileName = testName[i].split('.')[0]
        label = int(fileName.split('_')[0])
        vec = img2vector(validPath + "/" +  testName[i])
        label_pre = classify0(vec, traindata, trainlabel, 3)
        print(f"iter {i} the classifier came back with {label_pre}, the real answer is {label}")
        if label != label_pre:
            errorCnt += 1
    print(f"the total number of error is {errorCnt}")
    print(f"the total error rate is {errorCnt * 1.0 / m}")

In [18]:
headWritingClassfier()

iter 0 the classifier came back with 0, the real answer is 0
iter 1 the classifier came back with 0, the real answer is 0
iter 2 the classifier came back with 0, the real answer is 0
iter 3 the classifier came back with 0, the real answer is 0
iter 4 the classifier came back with 0, the real answer is 0
iter 5 the classifier came back with 0, the real answer is 0
iter 6 the classifier came back with 0, the real answer is 0
iter 7 the classifier came back with 0, the real answer is 0
iter 8 the classifier came back with 0, the real answer is 0
iter 9 the classifier came back with 0, the real answer is 0
iter 10 the classifier came back with 0, the real answer is 0
iter 11 the classifier came back with 0, the real answer is 0
iter 12 the classifier came back with 0, the real answer is 0
iter 13 the classifier came back with 0, the real answer is 0
iter 14 the classifier came back with 0, the real answer is 0
iter 15 the classifier came back with 0, the real answer is 0
iter 16 the classi

iter 132 the classifier came back with 1, the real answer is 1
iter 133 the classifier came back with 1, the real answer is 1
iter 134 the classifier came back with 1, the real answer is 1
iter 135 the classifier came back with 1, the real answer is 1
iter 136 the classifier came back with 1, the real answer is 1
iter 137 the classifier came back with 1, the real answer is 1
iter 138 the classifier came back with 1, the real answer is 1
iter 139 the classifier came back with 1, the real answer is 1
iter 140 the classifier came back with 1, the real answer is 1
iter 141 the classifier came back with 1, the real answer is 1
iter 142 the classifier came back with 1, the real answer is 1
iter 143 the classifier came back with 1, the real answer is 1
iter 144 the classifier came back with 1, the real answer is 1
iter 145 the classifier came back with 1, the real answer is 1
iter 146 the classifier came back with 1, the real answer is 1
iter 147 the classifier came back with 1, the real answ

iter 263 the classifier came back with 2, the real answer is 2
iter 264 the classifier came back with 2, the real answer is 2
iter 265 the classifier came back with 2, the real answer is 2
iter 266 the classifier came back with 2, the real answer is 2
iter 267 the classifier came back with 2, the real answer is 2
iter 268 the classifier came back with 2, the real answer is 2
iter 269 the classifier came back with 2, the real answer is 2
iter 270 the classifier came back with 2, the real answer is 2
iter 271 the classifier came back with 2, the real answer is 2
iter 272 the classifier came back with 2, the real answer is 2
iter 273 the classifier came back with 2, the real answer is 2
iter 274 the classifier came back with 2, the real answer is 2
iter 275 the classifier came back with 2, the real answer is 2
iter 276 the classifier came back with 3, the real answer is 3
iter 277 the classifier came back with 3, the real answer is 3
iter 278 the classifier came back with 3, the real answ

iter 395 the classifier came back with 4, the real answer is 4
iter 396 the classifier came back with 4, the real answer is 4
iter 397 the classifier came back with 4, the real answer is 4
iter 398 the classifier came back with 4, the real answer is 4
iter 399 the classifier came back with 4, the real answer is 4
iter 400 the classifier came back with 4, the real answer is 4
iter 401 the classifier came back with 4, the real answer is 4
iter 402 the classifier came back with 4, the real answer is 4
iter 403 the classifier came back with 4, the real answer is 4
iter 404 the classifier came back with 4, the real answer is 4
iter 405 the classifier came back with 4, the real answer is 4
iter 406 the classifier came back with 4, the real answer is 4
iter 407 the classifier came back with 4, the real answer is 4
iter 408 the classifier came back with 4, the real answer is 4
iter 409 the classifier came back with 4, the real answer is 4
iter 410 the classifier came back with 4, the real answ

iter 530 the classifier came back with 5, the real answer is 5
iter 531 the classifier came back with 5, the real answer is 5
iter 532 the classifier came back with 5, the real answer is 5
iter 533 the classifier came back with 5, the real answer is 5
iter 534 the classifier came back with 5, the real answer is 5
iter 535 the classifier came back with 5, the real answer is 5
iter 536 the classifier came back with 5, the real answer is 5
iter 537 the classifier came back with 5, the real answer is 5
iter 538 the classifier came back with 5, the real answer is 5
iter 539 the classifier came back with 5, the real answer is 5
iter 540 the classifier came back with 5, the real answer is 5
iter 541 the classifier came back with 5, the real answer is 5
iter 542 the classifier came back with 5, the real answer is 5
iter 543 the classifier came back with 5, the real answer is 5
iter 544 the classifier came back with 5, the real answer is 5
iter 545 the classifier came back with 5, the real answ

iter 663 the classifier came back with 6, the real answer is 6
iter 664 the classifier came back with 6, the real answer is 6
iter 665 the classifier came back with 6, the real answer is 6
iter 666 the classifier came back with 6, the real answer is 6
iter 667 the classifier came back with 6, the real answer is 6
iter 668 the classifier came back with 6, the real answer is 6
iter 669 the classifier came back with 6, the real answer is 6
iter 670 the classifier came back with 7, the real answer is 7
iter 671 the classifier came back with 7, the real answer is 7
iter 672 the classifier came back with 7, the real answer is 7
iter 673 the classifier came back with 7, the real answer is 7
iter 674 the classifier came back with 7, the real answer is 7
iter 675 the classifier came back with 7, the real answer is 7
iter 676 the classifier came back with 7, the real answer is 7
iter 677 the classifier came back with 7, the real answer is 7
iter 678 the classifier came back with 7, the real answ

iter 798 the classifier came back with 8, the real answer is 8
iter 799 the classifier came back with 8, the real answer is 8
iter 800 the classifier came back with 8, the real answer is 8
iter 801 the classifier came back with 8, the real answer is 8
iter 802 the classifier came back with 8, the real answer is 8
iter 803 the classifier came back with 8, the real answer is 8
iter 804 the classifier came back with 8, the real answer is 8
iter 805 the classifier came back with 8, the real answer is 8
iter 806 the classifier came back with 1, the real answer is 8
iter 807 the classifier came back with 8, the real answer is 8
iter 808 the classifier came back with 8, the real answer is 8
iter 809 the classifier came back with 8, the real answer is 8
iter 810 the classifier came back with 8, the real answer is 8
iter 811 the classifier came back with 8, the real answer is 8
iter 812 the classifier came back with 8, the real answer is 8
iter 813 the classifier came back with 8, the real answ

iter 931 the classifier came back with 9, the real answer is 9
iter 932 the classifier came back with 9, the real answer is 9
iter 933 the classifier came back with 9, the real answer is 9
iter 934 the classifier came back with 9, the real answer is 9
iter 935 the classifier came back with 9, the real answer is 9
iter 936 the classifier came back with 9, the real answer is 9
iter 937 the classifier came back with 9, the real answer is 9
iter 938 the classifier came back with 9, the real answer is 9
iter 939 the classifier came back with 9, the real answer is 9
iter 940 the classifier came back with 9, the real answer is 9
iter 941 the classifier came back with 9, the real answer is 9
iter 942 the classifier came back with 9, the real answer is 9
iter 943 the classifier came back with 9, the real answer is 9
iter 944 the classifier came back with 9, the real answer is 9
iter 945 the classifier came back with 9, the real answer is 9
the total number of error is 11
the total error rate is